# Enter Fisheries

In [ ]:
# from theano.sandbox import cuda
# cuda.use('gpu1')


In [25]:
%cd /Users/leahob/code/dl_class_nbs/lesson4
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

/Users/leahob/code/dl_class_nbs/lesson4


In [29]:
batch_size = 16

## Setup batches 

To do:
1. create validation set
2. create sample set
2. mode to separate dirs for each set
3. finetune and train

In [27]:
from __future__ import division,print_function
import math
import cPickle as pickle
import os, json
from glob import glob
import numpy as np
from matplotlib import pyplot as plt

import PIL
from PIL import Image
#import cv2
from numpy.random import random, permutation, randn, normal, uniform
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
import bcolz
import itertools

In [3]:
%cd "/Users/leahob/code/data/fisheries"

/Users/leahob/code/data/fisheries


In [4]:
%cd train

/Users/leahob/code/data/fisheries/train


### create validation set

In [5]:
%mkdir ../valid

In [10]:
for d in glob('*'):
    print(d)
    os.mkdir('../valid/'+d)

ALB
BET
DOL
LAG
NoF
OTHER
SHARK
YFT


In [11]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): os.rename(shuf[i], '../valid/'+shuf[i])

In [12]:
%ls -1 */*.jpg | cut -f1 -d"/" | sort | uniq -c

1536 ALB
 181 BET
 105 DOL
  57 LAG
 416 NoF
 268 OTHER
 154 SHARK
 660 YFT


In [14]:
 ls -1 ../valid/*/*.jpg | cut -f3 -d"/" | sort | uniq -c

 183 ALB
  19 BET
  12 DOL
  10 LAG
  49 NoF
  31 OTHER
  22 SHARK
  74 YFT


###  create sample

In [15]:
from shutil import copyfile

In [16]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [17]:
for d in glob('*'):
    print(d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../sample/train/'+d)

ALB
BET
DOL
LAG
NoF
OTHER
SHARK
YFT


In [18]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): copyfile(shuf[i], '../sample/train/'+shuf[i])

In [22]:
 ls -1 ../sample/train/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

 181 ALB
  27 BET
  18 DOL
   5 LAG
  38 NoF
  36 OTHER
  19 SHARK
  76 YFT


In [19]:
%cd ../valid
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(100): copyfile(shuf[i], '../sample/valid/'+shuf[i])

/Users/leahob/code/data/fisheries/valid


In [21]:
 ls -1 ../sample/valid/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

  47 ALB
   5 BET
   4 DOL
   3 LAG
  11 NoF
  14 OTHER
   2 SHARK
  14 YFT


### Create batches

In [32]:
path = '/Users/leahob/code/data/fisheries/sample/'
batches = get_batches(path+'train', batch_size=batch_size)

Found 400 images belonging to 8 classes.


In [33]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 100 images belonging to 8 classes.


### Imagenet conv features

- Small data set similar to imagenet images (full color photos)
- Use pre-trained VGG weights
- Precompute the output of the last convolutional layer

In [35]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

Exception: URL fetch failure on http://www.platform.ai/models/vgg16.h5: None -- [Errno 8] nodename nor servname provided, or not known

In [ ]:
conv_model = Sequential(conv_layers)